# load karaman-davis dataset (from kissim_app)

In [1]:
import sys
import os

# get the root path of the project adn add it to the system path
# This is necessary to import the modules from the project
root_path = os.path.abspath(os.path.join(os.getcwd(), "../../../kissim_app"))
print(root_path)
sys.path.append(root_path)


/Users/yanyz/Projects/kissim_mdfp/kissim_app


In [2]:
from src.data.kinases import kinmap
from src.data.profiling import load


karaman_davis_reduced = load('karaman-davis', pkidb_ligands=True, fda_approved=False)

Folder does not exist: /Users/yanyz/Projects/kissim_mdfp/kissim_app/src/../results
Use test folder instead: /Users/yanyz/Projects/kissim_mdfp/kissim_app/src/../test/results
Folder does not exist: /Users/yanyz/Projects/kissim_mdfp/kissim_app/src/../data/external/structures/20210902_KLIFS_HUMAN
Use test folder instead: /Users/yanyz/Projects/kissim_mdfp/kissim_app/src/../test/data/external/klifs_test


Changed ligand names (unknown names may be discarded - see function docstring):
           ligand.input            ligand.pkidb
0         Staurosporine  unknown (not in PKIDB)
1           JNJ-7706621  unknown (not in PKIDB)
2              MLN-8054  unknown (not in PKIDB)
3               PKC-412             Midostaurin
4              SU-14813  unknown (not in PKIDB)
5             SB-202190  unknown (not in PKIDB)
6             CP-724714  unknown (not in PKIDB)
7        VX-680/MK-0457  unknown (not in PKIDB)
9   Roscovitine/CYC-202  unknown (not in PKIDB)
10            SB-203580  unknown (not in PKIDB)
12            CP-690550             Tofacitinib
14              GW-2580  unknown (not in PKIDB)
16            GW-786034  unknown (not in PKIDB)
18               VX-745            Neflamapimod
19            SB-431542  unknown (not in PKIDB)
20              ZD-6474              Vandetanib
22         Flavopiridol               Alvocidib
24              MLN-518              Tandutinib
25      

In [3]:
kinase_names = ['ABL1', 'DDR1', 'EphA2', 'LOK', 'MYT1']

karaman_davis_reduced_subset = karaman_davis_reduced[karaman_davis_reduced.index.isin(kinase_names)]
df_karaman_davis = karaman_davis_reduced_subset[['Dasatinib', 'Bosutinib']]
df_karaman_davis

,Dasatinib,Bosutinib
ABL1,0.046,0.057
DDR1,0.690,120.000
EphA2,0.850,18.000
LOK,1200.000,7.000
MYT1,130.000,350.000


* LOK, ABL1; Bosutinib, Dasatinib
* LOK, EphA2; Bosutinib, Dasatinib
* LOK, MYT1; Bosutinib, Dasatinib
* ABL1, DDR1; Dasatinib, Imatinib
* ABL1, EphA2; Bosutinib, Dasatinib
* ABL1, MYT1; Bosutinib, Dasatinib
* EphA2, MYT1; Bosutinib, Dasatinib

# Fetch structures from klifs

In [4]:
from opencadd.databases.klifs import setup_remote

klifs_session = setup_remote()

In [5]:
# Get all available structures for these kinases
structures_df = klifs_session.structures.by_kinase_name(kinase_names=kinase_names)
print(f"Number of structures: {len(structures_df)}")
print("Kinases:", *structures_df["kinase.klifs_name"].unique())

Number of structures: 494
Kinases: MYT1 LOK ABL1 DDR1 EphA2


In [6]:
structures_df.columns

Index(['structure.klifs_id', 'structure.pdb_id', 'structure.alternate_model',
       'structure.chain', 'species.klifs', 'kinase.klifs_id',
       'kinase.klifs_name', 'kinase.names', 'kinase.family', 'kinase.group',
       'structure.pocket', 'ligand.expo_id', 'ligand_allosteric.expo_id',
       'ligand.klifs_id', 'ligand_allosteric.klifs_id', 'ligand.name',
       'ligand_allosteric.name', 'structure.dfg', 'structure.ac_helix',
       'structure.resolution', 'structure.qualityscore',
       'structure.missing_residues', 'structure.missing_atoms',
       'structure.rmsd1', 'structure.rmsd2', 'interaction.fingerprint',
       'structure.front', 'structure.gate', 'structure.back', 'structure.fp_i',
       'structure.fp_ii', 'structure.bp_i_a', 'structure.bp_i_b',
       'structure.bp_ii_in', 'structure.bp_ii_a_in', 'structure.bp_ii_b_in',
       'structure.bp_ii_out', 'structure.bp_ii_b', 'structure.bp_iii',
       'structure.bp_iv', 'structure.bp_v', 'structure.grich_distance',
       

## Filter structures

We filter the structures by different criteria:

- Species: human
- Conformation: DFG-in (the active kinase conformation)
- Resolution: <= 3 Angström
- Quality score*: >= 6

\* The KLIFS quality score takes into account the quality of the alignment, as well as the number of missing residues and atoms. A higher score indicates a better structure quality.

In [7]:
'''
structures_df = structures_df[
    (structures_df["species.klifs"] == "Human")
    & (structures_df["structure.dfg"] == "in")
    & (structures_df["structure.resolution"] <= 3)
    & (structures_df["structure.qualityscore"] >= 6)
]
print(f"Number of structures: {len(structures_df)}")
print("Kinases:", *structures_df["kinase.klifs_name"].unique())
print("Count the number of structures per kinase:", structures_df["kinase.klifs_name"].value_counts())
structures_df
'''

'\nstructures_df = structures_df[\n    (structures_df["species.klifs"] == "Human")\n    & (structures_df["structure.dfg"] == "in")\n    & (structures_df["structure.resolution"] <= 3)\n    & (structures_df["structure.qualityscore"] >= 6)\n]\nprint(f"Number of structures: {len(structures_df)}")\nprint("Kinases:", *structures_df["kinase.klifs_name"].unique())\nprint("Count the number of structures per kinase:", structures_df["kinase.klifs_name"].value_counts())\nstructures_df\n'

- 'Dasatinib' = 1N1
- 'Bosutinib' = DB8

In [8]:
# get all the data containing "ligand" in column name
df_dasatinib_bosutinib = structures_df[
    (structures_df['ligand.expo_id'] == '1N1') | (structures_df['ligand.expo_id'] == 'DB8')
]
df_dasatinib_bosutinib[['structure.klifs_id', 'structure.pdb_id', 'kinase.klifs_name', 'species.klifs', 'structure.dfg', 'structure.resolution', 'structure.qualityscore', 'ligand.expo_id']]

,structure.klifs_id,structure.pdb_id,kinase.klifs_name,species.klifs,structure.dfg,structure.resolution,structure.qualityscore,ligand.expo_id
11,8339,5vcv,MYT1,Human,in,1.92,8.0,1N1
17,8345,5vcy,MYT1,Human,in,1.56,8.0,DB8
18,8338,5vcy,MYT1,Human,in,1.56,8.0,DB8
24,948,5ajq,LOK,Human,in,2.20,7.6,DB8
25,940,5ajq,LOK,Human,in,2.20,7.6,DB8
26,955,5ajq,LOK,Human,in,2.20,9.6,DB8
52,950,5ajq,LOK,Human,in,2.20,9.6,DB8
55,8384,5owr,LOK,Human,in,2.30,6.4,1N1
56,8388,5owr,LOK,Human,in,2.30,6.4,1N1
92,1094,4xey,ABL1,Human,out-like,2.89,6.4,1N1


In [9]:
# for the same pdb, select the top 1 structure per kinase in terms of KLIFS quality score
df_dasatinib_bosutinib = df_dasatinib_bosutinib.sort_values('structure.qualityscore', ascending=False)
df_dasatinib_bosutinib = df_dasatinib_bosutinib.drop_duplicates(subset=['structure.pdb_id', 'kinase.klifs_name'])
df_dasatinib_bosutinib[['structure.klifs_id', 'structure.pdb_id', 'kinase.klifs_name', 'species.klifs', 'structure.dfg', 'structure.resolution', 'structure.qualityscore', 'ligand.expo_id']]

,structure.klifs_id,structure.pdb_id,kinase.klifs_name,species.klifs,structure.dfg,structure.resolution,structure.qualityscore,ligand.expo_id
26,955,5ajq,LOK,Human,in,2.20,9.6,DB8
147,1057,3ue4,ABL1,Human,out-like,2.42,8.9,DB8
266,10761,6bsd,DDR1,Human,out-like,2.61,8.4,1N1
11,8339,5vcv,MYT1,Human,in,1.92,8.0,1N1
17,8345,5vcy,MYT1,Human,in,1.56,8.0,DB8
401,7177,5i9x,EphA2,Human,in,1.43,8.0,DB8
392,7176,5i9y,EphA2,Human,in,1.23,8.0,1N1
219,1053,2gqg,ABL1,Human,in,2.40,8.0,1N1
103,14197,7n9g,ABL1,Human,out-like,2.20,8.0,1N1
92,1094,4xey,ABL1,Human,out-like,2.89,6.4,1N1


In [10]:
df_dasatinib_bosutinib[['kinase.klifs_name', 'ligand.expo_id']]

,kinase.klifs_name,ligand.expo_id
26,LOK,DB8
147,ABL1,DB8
266,DDR1,1N1
11,MYT1,1N1
17,MYT1,DB8
401,EphA2,DB8
392,EphA2,1N1
219,ABL1,1N1
103,ABL1,1N1
92,ABL1,1N1


## Merge Karaman-Davis bioactivity data into kissim df

In [11]:
df_karaman_davis_long = df_karaman_davis.rename(columns={'Dasatinib': '1N1', 'Bosutinib': 'DB8'})

df_karaman_davis_long = df_karaman_davis_long.reset_index().melt(id_vars='index', var_name='ligand.expo_id', value_name='kinmap.bioactivity')
df_karaman_davis_long = df_karaman_davis_long.rename(columns={'index': 'kinase.klifs_name'})

df_karaman_davis_long

,kinase.klifs_name,ligand.expo_id,kinmap.bioactivity
0,ABL1,1N1,0.046
1,DDR1,1N1,0.690
2,EphA2,1N1,0.850
3,LOK,1N1,1200.000
4,MYT1,1N1,130.000
5,ABL1,DB8,0.057
6,DDR1,DB8,120.000
7,EphA2,DB8,18.000
8,LOK,DB8,7.000
9,MYT1,DB8,350.000


In [12]:
df_merged = df_dasatinib_bosutinib.merge(df_karaman_davis_long, on=['kinase.klifs_name', 'ligand.expo_id'], how='left')
df_merged[['structure.klifs_id', 'structure.pdb_id', 'kinase.klifs_name', 'species.klifs', 'structure.dfg', 'structure.resolution', 'structure.qualityscore', 'ligand.expo_id', 'kinmap.bioactivity']]

,structure.klifs_id,structure.pdb_id,kinase.klifs_name,species.klifs,structure.dfg,structure.resolution,structure.qualityscore,ligand.expo_id,kinmap.bioactivity
0,955,5ajq,LOK,Human,in,2.20,9.6,DB8,7.000
1,1057,3ue4,ABL1,Human,out-like,2.42,8.9,DB8,0.057
2,10761,6bsd,DDR1,Human,out-like,2.61,8.4,1N1,0.690
3,8339,5vcv,MYT1,Human,in,1.92,8.0,1N1,130.000
4,8345,5vcy,MYT1,Human,in,1.56,8.0,DB8,350.000
5,7177,5i9x,EphA2,Human,in,1.43,8.0,DB8,18.000
6,7176,5i9y,EphA2,Human,in,1.23,8.0,1N1,0.850
7,1053,2gqg,ABL1,Human,in,2.40,8.0,1N1,0.046
8,14197,7n9g,ABL1,Human,out-like,2.20,8.0,1N1,0.046
9,1094,4xey,ABL1,Human,out-like,2.89,6.4,1N1,0.046


# Calculate KiSSim fingerprints

In [13]:
from kissim.api import encode
import numpy as np
import pandas as pd

In [14]:
structure_klifs_ids = df_dasatinib_bosutinib['structure.klifs_id']

kissim_fingerprints = encode(structure_klifs_ids)

In [15]:
kissim_fingerprints_array = [
    fingerprint.values_array().tolist()
    for _, fingerprint in kissim_fingerprints.data.items()
]
kissim_fingerprints_array = np.array(kissim_fingerprints_array)
kissim_fingerprints_df = pd.DataFrame(kissim_fingerprints_array, index=structure_klifs_ids)

print(f"Matrix shape: {kissim_fingerprints_df.shape}")
print(f"Number of fingerprints: {kissim_fingerprints_df.shape[0]}")
print(f"Number of fingerprint bits: {kissim_fingerprints_df.shape[1]}")
kissim_fingerprints_df

Matrix shape: (12, 1032)
Number of fingerprints: 12
Number of fingerprint bits: 1032


,0,1,2,3,4,5,6,7,8,9,...,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031
structure.klifs_id,,,,,,,,,,,,,,,,,,,,,
955,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,2.0,0.0,...,12.935410,11.943754,4.392296,4.712463,4.455708,3.494710,2.748480,2.987596,3.679778,1.538272
1057,2.0,1.0,1.0,0.0,1.0,0.0,3.0,3.0,2.0,1.0,...,13.025692,12.011242,4.460881,5.034019,4.077518,3.423783,2.929926,3.687617,3.166885,2.050392
10761,2.0,0.0,2.0,-1.0,0.0,0.0,NaN,3.0,2.0,1.0,...,12.857004,12.029064,4.471011,5.032515,4.133732,3.345548,3.054943,3.715677,2.721924,1.939075
8339,1.0,1.0,1.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,...,12.970246,11.975555,4.590457,4.656574,4.256000,3.443614,2.799831,3.435526,3.402649,1.802981
8345,1.0,1.0,1.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,...,12.979882,11.973692,4.545204,4.679642,4.239791,3.412097,2.645016,3.439387,3.357879,1.488037
7177,2.0,1.0,0.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,...,13.156184,12.171861,4.448386,4.729081,4.016259,3.348171,2.459051,3.189591,2.456419,1.516050
7176,2.0,1.0,0.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,...,13.144528,12.173165,4.458933,4.673591,4.044846,3.318169,2.554392,3.143976,2.470045,1.537785
1053,2.0,1.0,1.0,0.0,1.0,0.0,3.0,3.0,2.0,1.0,...,12.822959,11.962476,4.493169,4.836616,4.315794,3.468795,3.217910,3.302662,3.380991,2.067249
14197,2.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,2.0,1.0,...,12.813321,11.984674,4.472458,5.047574,4.216868,3.430281,2.944593,3.567419,2.978940,1.621562


# Compare structures

In [16]:
from sklearn.metrics.pairwise import nan_euclidean_distances

In [17]:
structure_distance_matrix_array = nan_euclidean_distances(kissim_fingerprints_df.values)

# Create DataFrame with structure KLIFS IDs as index/columns
structure_klifs_ids = kissim_fingerprints_df.index.to_list()
structure_distance_matrix_df = pd.DataFrame(
    structure_distance_matrix_array, index=structure_klifs_ids, columns=structure_klifs_ids
)
print(f"Structure distance matrix size: {structure_distance_matrix_df.shape}")
structure_distance_matrix_df

Structure distance matrix size: (12, 12)


,955,1057,10761,8339,8345,7177,7176,1053,14197,1094,8388,3254
955,0.000000,28.576142,29.268622,27.070272,25.880838,32.049004,32.338314,27.345969,26.987305,27.060062,20.127414,29.535628
1057,28.576142,0.000000,20.448719,24.263294,24.275118,22.898073,23.211865,17.856452,17.480245,8.191927,20.824617,20.386950
10761,29.268622,20.448719,0.000000,24.935382,26.078996,23.450612,23.192182,26.258210,27.292591,17.887989,23.689144,7.628437
8339,27.070272,24.263294,24.935382,0.000000,6.680548,25.098772,25.161640,25.959846,29.273259,23.767301,23.136211,24.507839
8345,25.880838,24.275118,26.078996,6.680548,0.000000,25.598288,25.751657,26.249652,28.653591,23.771006,23.395898,25.658868
7177,32.049004,22.898073,23.450612,25.098772,25.598288,0.000000,3.898270,27.826531,29.753711,19.909221,21.697771,22.961274
7176,32.338314,23.211865,23.192182,25.161640,25.751657,3.898270,0.000000,27.827668,29.961407,20.418159,22.328941,22.773133
1053,27.345969,17.856452,26.258210,25.959846,26.249652,27.826531,27.827668,0.000000,18.140601,16.351846,20.976468,25.701912
14197,26.987305,17.480245,27.292591,29.273259,28.653591,29.753711,29.961407,18.140601,0.000000,12.611501,22.060313,27.975786
1094,27.060062,8.191927,17.887989,23.767301,23.771006,19.909221,20.418159,16.351846,12.611501,0.000000,20.853383,18.341074


In [22]:
subset_labels = [8388, 1057, 1053, 955]
structure_distance_matrix_df.loc[subset_labels, subset_labels]

,8388,1057,1053,955
8388,0.000000,20.824617,20.976468,20.127414
1057,20.824617,0.000000,17.856452,28.576142
1053,20.976468,17.856452,0.000000,27.345969
955,20.127414,28.576142,27.345969,0.000000


# Map structure to kinase distance matrix¶

In [18]:
# Copy distance matrix to kinase matrix
kinase_distance_matrix_df = structure_distance_matrix_df.copy()
# Replace structure KLIFS IDs with the structures' kinase names
kinase_names = structures_df.set_index("structure.klifs_id").loc[
    structure_klifs_ids, "kinase.klifs_name"
]
kinase_distance_matrix_df.index = kinase_names
kinase_distance_matrix_df.columns = kinase_names
print("Show matrix subset:")
kinase_distance_matrix_df
# NBVAL_CHECK_OUTPUT


Show matrix subset:


kinase.klifs_name,LOK,ABL1,DDR1,MYT1,MYT1,EphA2,EphA2,ABL1,ABL1,ABL1,LOK,DDR1
kinase.klifs_name,,,,,,,,,,,,
LOK,0.000000,28.576142,29.268622,27.070272,25.880838,32.049004,32.338314,27.345969,26.987305,27.060062,20.127414,29.535628
ABL1,28.576142,0.000000,20.448719,24.263294,24.275118,22.898073,23.211865,17.856452,17.480245,8.191927,20.824617,20.386950
DDR1,29.268622,20.448719,0.000000,24.935382,26.078996,23.450612,23.192182,26.258210,27.292591,17.887989,23.689144,7.628437
MYT1,27.070272,24.263294,24.935382,0.000000,6.680548,25.098772,25.161640,25.959846,29.273259,23.767301,23.136211,24.507839
MYT1,25.880838,24.275118,26.078996,6.680548,0.000000,25.598288,25.751657,26.249652,28.653591,23.771006,23.395898,25.658868
EphA2,32.049004,22.898073,23.450612,25.098772,25.598288,0.000000,3.898270,27.826531,29.753711,19.909221,21.697771,22.961274
EphA2,32.338314,23.211865,23.192182,25.161640,25.751657,3.898270,0.000000,27.827668,29.961407,20.418159,22.328941,22.773133
ABL1,27.345969,17.856452,26.258210,25.959846,26.249652,27.826531,27.827668,0.000000,18.140601,16.351846,20.976468,25.701912
ABL1,26.987305,17.480245,27.292591,29.273259,28.653591,29.753711,29.961407,18.140601,0.000000,12.611501,22.060313,27.975786


In [19]:
# # We unstack the matrix (each pairwise comparison in a single row)
# We group by kinase names (level=[0, 1] ensures that the order of the kinases is ignored
# We take the minimum value in each kinase pair group
# We unstack the remaining data points
kinase_distance_matrix_df = (
    kinase_distance_matrix_df.unstack().groupby(level=[0, 1]).min().unstack(level=1)
)
# Cosmetics: Remove the index and column names
kinase_distance_matrix_df.index.name = None
kinase_distance_matrix_df.columns.name = None

print(
    f"Structure matrix of shape {structure_distance_matrix_df.shape} "
    f"reduced to kinase matrix of shape {kinase_distance_matrix_df.shape}."
)
# NBVAL_CHECK_OUTPUT

Structure matrix of shape (12, 12) reduced to kinase matrix of shape (5, 5).


In [20]:
# plot the distance matrix. lighter colors indicate similarity, darker colors dissimilarity.
import seaborn as sns

# Show matrix with background gradient
cm = sns.light_palette("green", as_cmap=True)
kinase_distance_matrix_df.style.background_gradient(cmap=cm).format("{:.3f}")

,ABL1,DDR1,EphA2,LOK,MYT1
ABL1,0.000,17.888,19.909,20.825,23.767
DDR1,17.888,0.000,22.773,23.689,24.508
EphA2,19.909,22.773,0.000,21.698,25.099
LOK,20.825,23.689,21.698,0.000,23.136
MYT1,23.767,24.508,25.099,23.136,0.000


## merge karaman-davis bioactivity data with kinase matrix